In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split

In [2]:
Ynames = ['fLength', 'fWidth', 'fSize', 'fConc', 'fConc1', 'fAsym', 
             'fM3Long', 'fM3Trans', 'fAlpha', 'fDist', 'identity']
featureNames = ['fLength', 'fWidth', 'fSize', 'fConc', 'fConc1', 'fAsym', 
             'fM3Long', 'fM3Trans', 'fAlpha', 'fDist']
filepath = 'data/magic04.data'
data = pd.read_csv(filepath, names=Ynames, header=None)
data['identity']= data['identity'].map({'g':1, 'h': 0}).astype(int) 
X = data[featureNames].values
Y = data['identity'].values.astype('int64')

In [3]:
minmax_scale = preprocessing.MinMaxScaler(feature_range=(0, 1))
X=minmax_scale.fit_transform(X)
train_Features, test_Features, train_Label, test_Label = train_test_split(X, Y, test_size=0.33, random_state=None) 
train_Features[:2]

array([[0.5233583 , 0.13235016, 0.39908338, 0.17820207, 0.13113054,
        0.32151373, 0.24748597, 0.57926608, 0.61378333, 0.52699531],
       [0.05560825, 0.04231654, 0.08166765, 0.59813615, 0.41709883,
        0.46583607, 0.61526659, 0.55917953, 0.62474444, 0.37872058]])

In [4]:
test_Features[:2]

array([[0.05773015, 0.05013222, 0.08397398, 0.61427435, 0.42732257,
        0.42668189, 0.56542385, 0.51560004, 0.00736667, 0.36047377],
       [0.12392605, 0.04782317, 0.29858072, 0.31753608, 0.27218847,
        0.50138314, 0.68008581, 0.51891129, 0.34476   , 0.38705211]])

In [5]:
train_Label[:4]

array([0, 1, 1, 1])

In [6]:
from keras.utils import np_utils
train_Label = train_Label.reshape(train_Label.shape[0], 1)
test_Label= test_Label.reshape(test_Label.shape[0], 1)
train_Label=np_utils.to_categorical(train_Label)
test_Label=np_utils.to_categorical(test_Label)

Using TensorFlow backend.


In [7]:
train_Features.shape

(12743, 10)

In [8]:
test_Label[:4]

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [9]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D

In [10]:
train_Features = train_Features.reshape(train_Features.shape[0], 10, 1, 1)

In [11]:
model = Sequential()

In [12]:
model.add(Conv2D(filters=1000,
                 kernel_size=(1,1),
                 padding='valid',
                 input_shape=(10,1,1), 
                 activation='relu'))

In [13]:
model.add(Conv2D(filters=1000,
                 kernel_size=(1,1),
                 padding='valid',
                 activation='relu'))

In [14]:
model.add(MaxPooling2D(pool_size=(1, 1)))

In [15]:
model.add(Dropout(0.25))

In [16]:
model.add(Flatten())

In [17]:
model.add(Dense(128, activation='tanh'))

In [18]:
model.add(Dropout(0.25))

In [19]:
model.add(Dense(2,activation='sigmoid'))

In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 10, 1, 1000)       2000      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 1, 1000)       1001000   
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 1, 1000)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 1, 1000)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1280128   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [21]:
model.compile(loss='categorical_crossentropy',
              optimizer='Nadam',metrics=['accuracy']) 


In [22]:
model.fit(x=train_Features,y=train_Label,batch_size=1000)

Epoch 1/1
12743/12743 [==============================] - 91s 7ms/step - loss: 0.6234 - acc: 0.6980


In [23]:
test_Features = test_Features.reshape(test_Features.shape[0], 10, 1, 1)

In [24]:
scores = model.evaluate(x=test_Features, 
                        y=test_Label)

6277/6277 [==============================] - 13s 2ms/step


In [25]:
scores[1]

0.740799745101163